In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install h3
!pip install shapely
!pip install geopy
!pip install geopandas
!pip install folium
!pip install h5py

     |████████████████████████████████| 793 kB 4.3 MB/s 
     |████████████████████████████████| 1.0 MB 4.1 MB/s 
     |████████████████████████████████| 6.3 MB 51.5 MB/s 
     |████████████████████████████████| 16.7 MB 50.8 MB/s 


In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import h3
import h3.api.basic_int as h3_int
import h3.api.numpy_int as h3_np
import h3.api.memview_int as h3_mv
from shapely.geometry import Point, Polygon
from geopy.distance import distance
import geopandas as gpd
import h5py

In [4]:
gps = pd.read_csv('/content/drive/MyDrive/data/bike/bike_gps1.csv')
gps_OD = pd.read_csv('/content/drive/MyDrive/data/bike/GPS_OD.csv')

In [28]:
data = gps.drop(['Unnamed: 0'], axis=1)

In [29]:
data.head()

,rowId,trackingNo,latitude,longitude,useId,str_100m
0,0,1,37.526988,126.932311,1,37.527 126.932
1,1,1,37.517536,126.864296,2,37.518 126.864
2,2,2,37.517391,126.864349,2,37.517 126.864
3,3,3,37.517714,126.864510,2,37.518 126.865
4,4,4,37.517574,126.864510,2,37.518 126.865


In [30]:
gps_OD.head()

,군집개수,대여소ID,대여소명,gu,address,lat,lon,odate,nracks,optype,cat1,cat2,ndx,kw,kk,str_100m,h3_9,h3_10
0,600 new,1072,고덕역 5번출구,강동구,명일동,37.5549,127.1537,2020.1.13,10,QR,교통,지하철역,790,262,285,37.555 127.154,8930e1ce293ffff,8a30e1ce2907fff
1,600 new,4854,가락시장역 2번 출구,송파구,가락동 184-23,37.4927,127.1180,2021.1.4,10,QR,교통,지하철역,2472,300,514,37.493 127.118,8930e1c8d13ffff,8a30e1c8d11ffff
2,600 new,4860,가락쌍용2차아파트 103동,송파구,가락동 22-7,37.5023,127.1220,2021.4.13,8,QR,주거지,아파트,2477,576,398,37.502 127.122,8930e1c8db7ffff,8a30e1c8db67fff
3,600 new,2626,경찰병원,송파구,가락동 59,37.4957,127.1239,2020.1.20,15,QR,쇼핑,병원,1795,317,397,37.496 127.124,8930e1c8dabffff,8a30e1c8da8ffff
4,600 new,2658,가락몰 업무동,송파구,가락동 600,37.4954,127.1157,2020.6.4,10,QR,쇼핑,상가,1825,300,514,37.495 127.116,8930e1c8d8bffff,8a30e1c8d88ffff


In [31]:
lat = gps_OD['lat'].tolist()
lon = gps_OD['lon'].tolist()

gpsRound = []

for i in tqdm(range(len(gps_OD))):
  gpsRound.append(f'{round(lat[i], 3)} {round(lon[i], 3)}')

gps_OD['str_100m'] = gpsRound

100%|██████████| 2485/2485 [00:00<00:00, 194112.03it/s]


In [32]:
a = gps_OD['str_100m'].tolist()
print(f'대여소 개수 : {len(a)}')
print(f'대여소 개수(Unique) : {len(set(a))}')

대여소 개수 : 2485
대여소 개수(Unique) : 2391


In [33]:
statName = gps_OD['대여소명'].tolist()
mappingDict = {concat : name for concat, name in zip(gpsRound, statName)}

In [34]:
concat = data['str_100m'].tolist()
stat = []
for i in range(len(concat)):
  stat.append(mappingDict.get(concat[i], ''))
data['대여소'] = stat

In [35]:
stat.count('')

4983822

In [36]:
data.head(100)

,rowId,trackingNo,latitude,longitude,useId,str_100m,대여소
0,0,1,37.526988,126.932311,1,37.527 126.932,여의나루역 1번출구 앞
1,1,1,37.517536,126.864296,2,37.518 126.864,
2,2,2,37.517391,126.864349,2,37.517 126.864,신정6동 주민센터 인근
3,3,3,37.517714,126.864510,2,37.518 126.865,
4,4,4,37.517574,126.864510,2,37.518 126.865,
...,...,...,...,...,...,...,...
95,95,2,37.568348,126.825348,11,37.568 126.825,
96,96,3,37.568328,126.825118,11,37.568 126.825,
97,97,4,37.568016,126.825263,11,37.568 126.825,
98,98,5,37.568241,126.825156,11,37.568 126.825,


In [37]:
lat = gps_OD['lat'].tolist()
lon = gps_OD['lon'].tolist()
resolution_9 = []
resolution_10 = []

for i in range(len(gps_OD)):
    grid_9 = h3.geo_to_h3(lat[i], lon[i], 9)
    grid_10 = h3.geo_to_h3(lat[i], lon[i], 10)
    
    resolution_9.append(grid_9)
    resolution_10.append(grid_10)

gps_OD['h3_9'] = resolution_9
gps_OD['h3_10'] = resolution_10

In [38]:
statName = gps_OD['대여소명'].tolist()
mappingDict_h3_9 = {grid : name for grid, name in zip(resolution_9, statName)}
mappingDict_h3_10 = {grid : name for grid, name in zip(resolution_10, statName)}

In [39]:
print(f'h3_9 대여소 개수 : {len(mappingDict_h3_9)}')
print(f'h3_10 대여소 개수 : {len(mappingDict_h3_10)}')

h3_9 대여소 개수 : 1957
h3_10 대여소 개수 : 2376


In [40]:
lat = data['latitude'].tolist()
lon = data['longitude'].tolist()
gps_resolution_9 = []
gps_resolution_10 = []

for i in range(len(data)):
    grid_9 = h3.geo_to_h3(lat[i], lon[i], 9)
    grid_10 = h3.geo_to_h3(lat[i], lon[i], 10)
    
    gps_resolution_9.append(grid_9)
    gps_resolution_10.append(grid_10)

data['h3_9'] = gps_resolution_9
data['h3_10'] = gps_resolution_10

In [41]:
h3_9 = data['h3_9'].tolist()
h3_10 = data['h3_10'].tolist()

stat_9 = []
stat_10 = []

for i in range(len(data)):
  stat_9.append(mappingDict_h3_9.get(h3_9[i], ''))
  stat_10.append(mappingDict_h3_10.get(h3_10[i], ''))

data['대여소_h3_9'] = stat_9
data['대여소_h3_10'] = stat_10

In [42]:
print(stat_9.count(''))
print(stat_10.count(''))

3049156
4977097


In [43]:
data.head(100)

,rowId,trackingNo,latitude,longitude,useId,str_100m,대여소,h3_9,h3_10,대여소_h3_9,대여소_h3_10
0,0,1,37.526988,126.932311,1,37.527 126.932,여의나루역 1번출구 앞,8930e1d9527ffff,8a30e1d9525ffff,여의나루역 1번출구 앞,여의나루역 1번출구 앞
1,1,1,37.517536,126.864296,2,37.518 126.864,,8930e0acd47ffff,8a30e0acd477fff,"양천구청, 보건소 사잇길",신정6동 주민센터 인근
2,2,2,37.517391,126.864349,2,37.517 126.864,신정6동 주민센터 인근,8930e0acd47ffff,8a30e0acd477fff,"양천구청, 보건소 사잇길",신정6동 주민센터 인근
3,3,3,37.517714,126.864510,2,37.518 126.865,,8930e0acd47ffff,8a30e0acd477fff,"양천구청, 보건소 사잇길",신정6동 주민센터 인근
4,4,4,37.517574,126.864510,2,37.518 126.865,,8930e0acd47ffff,8a30e0acd477fff,"양천구청, 보건소 사잇길",신정6동 주민센터 인근
...,...,...,...,...,...,...,...,...,...,...,...
95,95,2,37.568348,126.825348,11,37.568 126.825,,8930e0aedd7ffff,8a30e0aedc2ffff,마곡엠밸리4단지 정문,
96,96,3,37.568328,126.825118,11,37.568 126.825,,8930e0aedd7ffff,8a30e0aedc2ffff,마곡엠밸리4단지 정문,
97,97,4,37.568016,126.825263,11,37.568 126.825,,8930e0aedc3ffff,8a30e0aedc2ffff,,
98,98,5,37.568241,126.825156,11,37.568 126.825,,8930e0aedc3ffff,8a30e0aedc2ffff,,


In [44]:
data = data.drop(['h3_9', 'h3_10'], axis=1)

In [46]:
data.to_csv('/content/drive/MyDrive/data/bike/bike_gps1_v1.csv', index = False)

In [61]:
num = data['trackingNo'].tolist()
print(f'대여 시작 지점 개수 : {num.count(1)}')

count = 0
for i in (range(len(num))):
  if num[i] == 1 and stat_9[i]:
    count += 1
print(f'대여 시작점 중 Mapping 지점 : {count}')

대여 시작 지점 개수 : 267553
대여 시작점 중 Mapping 지점 : 215270


In [62]:
data.head(100)

,rowId,trackingNo,latitude,longitude,useId,str_100m,대여소,대여소_h3_9,대여소_h3_10
0,0,1,37.526988,126.932311,1,37.527 126.932,여의나루역 1번출구 앞,여의나루역 1번출구 앞,여의나루역 1번출구 앞
1,1,1,37.517536,126.864296,2,37.518 126.864,,"양천구청, 보건소 사잇길",신정6동 주민센터 인근
2,2,2,37.517391,126.864349,2,37.517 126.864,신정6동 주민센터 인근,"양천구청, 보건소 사잇길",신정6동 주민센터 인근
3,3,3,37.517714,126.864510,2,37.518 126.865,,"양천구청, 보건소 사잇길",신정6동 주민센터 인근
4,4,4,37.517574,126.864510,2,37.518 126.865,,"양천구청, 보건소 사잇길",신정6동 주민센터 인근
...,...,...,...,...,...,...,...,...,...
95,95,2,37.568348,126.825348,11,37.568 126.825,,마곡엠밸리4단지 정문,
96,96,3,37.568328,126.825118,11,37.568 126.825,,마곡엠밸리4단지 정문,
97,97,4,37.568016,126.825263,11,37.568 126.825,,,
98,98,5,37.568241,126.825156,11,37.568 126.825,,,
